In [1]:
%%bash
pip install pyvo astroquery astropy

In [13]:
from astroquery.simbad import Simbad
from astropy.coordinates import Angle
import astropy.units as u

def get_celestial_info(object_name):
    customSimbad = Simbad()
    customSimbad.add_votable_fields('dimensions')

    result = customSimbad.query_object(object_name)

    if result is None:
        return "Object not found."

    # Extract coordinates
    ra = result['RA'].data[0]
    dec = result['DEC'].data[0]

    # Convert to Angle objects for better representation
    ra = Angle(f"{ra} hours")
    dec = Angle(f"{dec} degrees")
    print(result.columns)
    # Extract dimensions (angular size)
    dimensions = result['DIMENSIONS'].data[0]
    if dimensions is not None:
        # Assume the first dimension is the major axis
        major_axis = Angle(dimensions.split('x')[0] + " arcmin")
    else:
        major_axis = None

    return ra, dec, major_axis

# Example usage
object_name = "NGC 7000"
ra, dec, major_axis = get_celestial_info(object_name)

print(f"Object: {object_name}")
print(f"RA: {ra.to_string(u.hour)}")
print(f"DEC: {dec.to_string(u.deg)}")
if major_axis:
    print(f"Major Axis: {major_axis.to_string(u.arcmin)}")
else:
    print("Major Axis: Not available")


<TableColumns names=('MAIN_ID','RA','DEC','RA_PREC','DEC_PREC','COO_ERR_MAJA','COO_ERR_MINA','COO_ERR_ANGLE','COO_QUAL','COO_WAVELENGTH','COO_BIBCODE','GALDIM_MAJAXIS','GALDIM_MINAXIS','GALDIM_ANGLE','GALDIM_QUAL','GALDIM_WAVELENGTH','GALDIM_BIBCODE','SCRIPT_NUMBER_ID')>


KeyError: 'DIMENSIONS'

In [ ]:
import pyvo as vo
from astropy.coordinates import SkyCoord
import astropy.units as u

service = vo.dal.TAPService("http://tapvizier.u-strasbg.fr/TAPVizieR/tap")

def description_and_examples():
    if hasattr(service, 'description'):
        print(service.description)
    if hasattr(service, 'examples'):
        print(service.examples)
        
def find_table_by_keyword(keyword:str):
    for table in service.tables:
        if keyword.lower() in table.description.lower():
            print(table.name, ":", table.description)


def table_names_and_descriptions():
    for table in service.tables:
        print(f"{table.name}: {table.description}")

def table_columns():
    # Print out the available tables and columns
    for table in service.tables:
        print(f"Table name: {table.name}")
        for column in table.columns:
            print(f"  - {column.name}")

def get_celestial_info_with_pyvo(object_name):
    # SQL query to find the object
    query = f"""
    SELECT TOP 1 RAJ2000, DEJ2000, BmajAxis, BminAxis
    FROM "II/246/out"
    WHERE MAIN_ID = '{object_name}'
    """
    query = f"""DESCRIBE TABLE II/246/out"""

    # Execute the query
    result = service.search(query)

    if len(result) == 0:
        return "Object not found."

    print(result)
    # Fetch RA, DEC, and angular size (BmajAxis) from the result
    ra = result['RAJ2000'][0] * u.deg
    dec = result['DEJ2000'][0] * u.deg
    major_axis = result['BmajAxis'][0] * u.arcsec  # Assuming major axis is in arcsec

    # Convert RA, DEC to SkyCoord object for easier manipulation
    coord = SkyCoord(ra, dec, frame='icrs')

    return coord.ra, coord.dec, major_axis

# description_and_examples()
find_table_by_keyword('DSS')
# table_names_and_descriptions()
# table_columns()

In [15]:

# Example usage
object_name = "M31"
ra, dec, major_axis = get_celestial_info_with_pyvo(object_name)

if ra:
    print(f"Object: {object_name}")
    print(f"RA: {ra.to_string(u.hour)}")
    print(f"DEC: {dec.to_string(u.deg)}")
    print(f"Major Axis: {major_axis.to_string(u.arcmin)}")
else:
    print("Object not found.")


J/A+A/635/A13/table1: Radial velocities and activity indicators ( Faria J.P., Adibekyan V., Amazo-Gomez E.M., Barros S.C.C., Camacho J.D., Demangeon O., Figueira P., Mortier A., Oshagh M., Pepe F., Santos N.C., Gomes da Silva J., Costa Silva A.R., Sousa S.G., Ulmer-Moll S., Viana P.T.P.)
J/A+A/665/A28/table1: Average ULX spectra ( Kovlakas K., Fragos T., Schaerer D., Mesinger A.)
J/A+A/674/A48/fgas-esz: Planck-ESZ gas fraction sample ( Wicker R., Douspis M., Salvati L., Aghanim N.)
J/A+A/653/A133/te: Transition East catalog ( Nogueras-Lara F., Schoedel R., Neumayer N.)
J/A+A/456/985/table5: Radial Velocities for M31 globular clusters ( Galleti S., Federici L., Bellazzini M., Buzzoni A., Fusi Pecci F.)
J/A+A/649/A31/list: Information on datacube ( Herrera-Camus R., Foerster Schreiber N., Genzel R., Tacconi L., Bolatto A., Davies R.L., Fisher D., Lutz D., Naab T., Shimizu T., Tadaki K., Uebler H.)
J/A+A/433/745/table1: Multiconfiguration Dirac-Fock (MCDF) calculated energy levels relativ

KeyboardInterrupt: 